In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./Churn_Modelling.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [5]:
X = df.drop(['RowNumber','CustomerId','Surname','Exited'], axis = 1).values


In [6]:
y = df['Exited'].values

In [7]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ..., 
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

# Data preprocessing


In [8]:
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder

In [9]:
labelencoder_X = LabelEncoder()

In [10]:
X[:,2] = labelencoder_X.fit_transform(X[:,2])

In [11]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ..., 
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [12]:
X[:,1] = labelencoder_X.fit_transform(X[:,1])

In [13]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ..., 
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [14]:
X.shape


(10000, 10)

In [15]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ..., 
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [16]:
onehotencoder = OneHotEncoder(categorical_features =[1])

In [17]:
X = onehotencoder.fit_transform(X).toarray()

In [18]:
X = X[:,1:]

In [19]:
X.shape

(10000, 11)

In [20]:
#Splitting data into train and test set
from sklearn.model_selection import train_test_split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 0)

In [21]:
#Scaling data

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# CLASSIFICATION



In [22]:
from keras.layers import Dense
from keras.optimizers import Adam
from keras.models import Sequential
import keras.backend as K


Using TensorFlow backend.


In [44]:
#Model builder function
def build_model():
    model = Sequential()
    model.add(Dense(20, input_dim = 11,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(10,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(10,kernel_initializer = 'uniform', activation = 'relu'))
    model.add(Dense(1,kernel_initializer = 'uniform', activation = 'sigmoid'))
    model.compile(optimizer = 'adam',loss='binary_crossentropy', metrics = ['accuracy'])
    return model


In [45]:
#Cross validiation step
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score


In [46]:
K.clear_session()

In [47]:

model = KerasClassifier(build_fn = build_model,batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = model, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()


Epoch 1/100
Epoch 1/100
Epoch 1/100
6810/7200 [===========================>..] - ETA: 0s - loss: 0.4667 - acc: 0.7981
Epoch 2/100
7200/7200 [==============================] - 4s - loss: 0.4646 - acc: 0.7969     
Epoch 2/100
  10/7200 [..............................] - ETA: 3s - loss: 0.4443 - acc: 0.9000
Epoch 2/100
7200/7200 [==============================] - 3s - loss: 0.4205 - acc: 0.8035     
Epoch 3/100
 650/7200 [=>............................] - ETA: 3s - loss: 0.3816 - acc: 0.8508
Epoch 3/100
 940/7200 [==>...........................] - ETA: 3s - loss: 0.3877 - acc: 0.8457
Epoch 3/100
7200/7200 [==============================] - 4s - loss: 0.4158 - acc: 0.8254     
Epoch 4/100
6380/7200 [=========================>....] - ETA: 0s - loss: 0.4229 - acc: 0.8197
Epoch 4/100
1130/7200 [===>..........................] - ETA: 3s - loss: 0.4053 - acc: 0.8230
Epoch 4/100
6030/7200 [========================>.....] - ETA: 0s - loss: 0.4200 - acc: 0.8234
Epoch 5/100
1550/7200 [=====>.......

3180/7200 [============>.................] - ETA: 2s - loss: 0.4069 - acc: 0.8343
Epoch 18/100
5210/7200 [====================>.........] - ETA: 1s - loss: 0.4100 - acc: 0.8294
Epoch 18/100
1890/7200 [======>.......................] - ETA: 3s - loss: 0.4025 - acc: 0.8386
Epoch 19/100
7200/7200 [==============================] - 3s - loss: 0.3992 - acc: 0.8379     
Epoch 19/100
7200/7200 [==============================] - 3s - loss: 0.4019 - acc: 0.8362     
Epoch 20/100
7200/7200 [==============================] - 3s - loss: 0.4012 - acc: 0.8376     
Epoch 20/100
7200/7200 [==============================] - 3s - loss: 0.3993 - acc: 0.8361     
Epoch 20/100
4300/7200 [================>.............] - ETA: 1s - loss: 0.4021 - acc: 0.8344
Epoch 21/100
6450/7200 [=========================>....] - ETA: 0s - loss: 0.3986 - acc: 0.8371Epoch 21/100

3630/7200 [==============>...............] - ETA: 1s - loss: 0.4061 - acc: 0.8278
Epoch 21/100
3970/7200 [===============>..............] - ETA: 

7200/7200 [==============================] - 3s - loss: 0.3964 - acc: 0.8386     
Epoch 34/100
 920/7200 [==>...........................] - ETA: 2s - loss: 0.3947 - acc: 0.8348
Epoch 34/100
7200/7200 [==============================] - 3s - loss: 0.3955 - acc: 0.8375     
Epoch 35/100
7200/7200 [==============================] - 3s - loss: 0.3414 - acc: 0.8646     
Epoch 35/100
7200/7200 [==============================] - 3s - loss: 0.3902 - acc: 0.8372     
Epoch 35/100
2690/7200 [==========>...................] - ETA: 2s - loss: 0.3863 - acc: 0.8409
Epoch 35/100
7200/7200 [==============================] - 3s - loss: 0.3961 - acc: 0.8372     
Epoch 36/100
7200/7200 [==============================] - 3s - loss: 0.3404 - acc: 0.8647     
Epoch 36/100
 360/7200 [>.............................] - ETA: 4s - loss: 0.3060 - acc: 0.8639
Epoch 36/100
7200/7200 [==============================] - 3s - loss: 0.3915 - acc: 0.8387     
Epoch 36/100
6910/7200 [===========================>..] - ETA: 

3240/7200 [============>.................] - ETA: 1s - loss: 0.3438 - acc: 0.8605
Epoch 50/100
7200/7200 [==============================] - 3s - loss: 0.3348 - acc: 0.8654     
Epoch 50/100
5620/7200 [======================>.......] - ETA: 0s - loss: 0.3306 - acc: 0.8662
Epoch 50/100
7200/7200 [==============================] - 3s - loss: 0.3846 - acc: 0.8424     
Epoch 51/100
4640/7200 [==================>...........] - ETA: 1s - loss: 0.3838 - acc: 0.8444
Epoch 52/100
7200/7200 [==============================] - 3s - loss: 0.3335 - acc: 0.8640     
Epoch 52/100
7200/7200 [==============================] - 3s - loss: 0.3849 - acc: 0.8425     Epoch 52/100

7200/7200 [==============================] - 3s - loss: 0.3331 - acc: 0.8657     
Epoch 52/100
2770/7200 [==========>...................] - ETA: 2s - loss: 0.3457 - acc: 0.8552
Epoch 53/100
7200/7200 [==============================] - 4s - loss: 0.3851 - acc: 0.8397     
Epoch 53/100
7200/7200 [==============================] - 3s - 

2810/7200 [==========>...................] - ETA: 2s - loss: 0.3253 - acc: 0.8687
Epoch 66/100
7200/7200 [==============================] - 3s - loss: 0.3310 - acc: 0.8640     
Epoch 67/100
7200/7200 [==============================] - 3s - loss: 0.3259 - acc: 0.8682     
Epoch 67/100
7200/7200 [==============================] - 3s - loss: 0.3287 - acc: 0.8675     
Epoch 67/100
4740/7200 [==================>...........] - ETA: 1s - loss: 0.3318 - acc: 0.8654
Epoch 68/100
4970/7200 [===================>..........] - ETA: 1s - loss: 0.3308 - acc: 0.8654
Epoch 68/100
2440/7200 [=========>....................] - ETA: 2s - loss: 0.3305 - acc: 0.8676
Epoch 68/100
7200/7200 [==============================] - 3s - loss: 0.3406 - acc: 0.8604     
Epoch 68/100
7200/7200 [==============================] - 3s - loss: 0.3300 - acc: 0.8665     
Epoch 69/100
7200/7200 [==============================] - 3s - loss: 0.3284 - acc: 0.8667     
Epoch 69/100
7200/7200 [==============================] - 3s - 

...

7200/7200 [==============================] - 4s - loss: 0.3286 - acc: 0.8678     
Epoch 61/100
7200/7200 [==============================] - 4s - loss: 0.3404 - acc: 0.8624     
Epoch 62/100
5640/7200 [======================>.......] - ETA: 0s - loss: 0.3240 - acc: 0.8681
Epoch 62/100
 660/7200 [=>............................] - ETA: 3s - loss: 0.3035 - acc: 0.8742
Epoch 62/100
4490/7200 [=================>............] - ETA: 1s - loss: 0.3226 - acc: 0.8673
Epoch 63/100
7200/7200 [==============================] - 3s - loss: 0.3334 - acc: 0.8635     
Epoch 63/100
6520/7200 [==========================>...] - ETA: 0s - loss: 0.3281 - acc: 0.8661
Epoch 63/100
7200/7200 [==============================] - 4s - loss: 0.3398 - acc: 0.8625     
Epoch 64/100
7200/7200 [==============================] - 3s - loss: 0.3277 - acc: 0.8656     
Epoch 64/100
7200/7200 [==============================] - 4s - loss: 0.3394 - acc: 0.8611     Epoch 65/100

7200/7200 [==============================] - 3s - 

5070/7200 [====================>.........] - ETA: 1s - loss: 0.3278 - acc: 0.8645
Epoch 78/100
4500/7200 [=================>............] - ETA: 1s - loss: 0.3384 - acc: 0.8609
Epoch 78/100
4990/7200 [===================>..........] - ETA: 1s - loss: 0.3344 - acc: 0.8629
Epoch 78/100
7200/7200 [==============================] - 3s - loss: 0.3312 - acc: 0.8646     
Epoch 79/100
7200/7200 [==============================] - 3s - loss: 0.3234 - acc: 0.8685     
Epoch 79/100
1450/7200 [=====>........................] - ETA: 2s - loss: 0.3571 - acc: 0.8510
Epoch 79/100
7200/7200 [==============================] - 3s - loss: 0.3377 - acc: 0.8619     
Epoch 80/100
7200/7200 [==============================] - 4s - loss: 0.3268 - acc: 0.8657     
Epoch 80/100
 880/7200 [==>...........................] - ETA: 3s - loss: 0.3231 - acc: 0.8682
Epoch 81/100
7200/7200 [==============================] - 4s - loss: 0.3248 - acc: 0.8687     
Epoch 82/100
7200/7200 [==============================] - 4s - 

1000/7200 [===>..........................] - ETA: 3s - loss: 0.3007 - acc: 0.8790
Epoch 93/100
2800/7200 [==========>...................] - ETA: 2s - loss: 0.3257 - acc: 0.8679
Epoch 93/100
7200/7200 [==============================] - 5s - loss: 0.3368 - acc: 0.8639     
Epoch 94/100
2710/7200 [==========>...................] - ETA: 2s - loss: 0.3055 - acc: 0.8775
Epoch 95/100
1070/7200 [===>..........................] - ETA: 3s - loss: 0.3247 - acc: 0.8617
Epoch 95/100
7200/7200 [==============================] - 4s - loss: 0.3252 - acc: 0.8681     
Epoch 95/100
2320/7200 [========>.....................] - ETA: 3s - loss: 0.3179 - acc: 0.8746
Epoch 96/100
 530/7200 [=>............................] - ETA: 2s - loss: 0.3198 - acc: 0.8660
Epoch 96/100
4210/7200 [================>.............] - ETA: 1s - loss: 0.3243 - acc: 0.8667
Epoch 96/100
7200/7200 [==============================] - 4s - loss: 0.3217 - acc: 0.8689     
Epoch 97/100
7200/7200 [==============================] - 4s - 

7200/7200 [==============================] - 2s - loss: 0.3367 - acc: 0.8625     
Epoch 21/100
7200/7200 [==============================] - 2s - loss: 0.3487 - acc: 0.8604     
Epoch 21/100
7200/7200 [==============================] - 2s - loss: 0.3359 - acc: 0.8629     
Epoch 22/100
7200/7200 [==============================] - 2s - loss: 0.3484 - acc: 0.8625     
Epoch 22/100
7200/7200 [==============================] - 2s - loss: 0.3478 - acc: 0.8624     
Epoch 23/100
7200/7200 [==============================] - 2s - loss: 0.3356 - acc: 0.8631     
Epoch 24/100
7200/7200 [==============================] - 2s - loss: 0.3466 - acc: 0.8585     
Epoch 24/100
7200/7200 [==============================] - 2s - loss: 0.3347 - acc: 0.8654     
Epoch 25/100
7200/7200 [==============================] - 2s - loss: 0.3473 - acc: 0.8633     
Epoch 25/100
7200/7200 [==============================] - 2s - loss: 0.3353 - acc: 0.8637     
Epoch 26/100
7200/7200 [==============================] - 2s - 

7200/7200 [==============================] - 2s - loss: 0.3268 - acc: 0.8654     
Epoch 61/100
7200/7200 [==============================] - 2s - loss: 0.3369 - acc: 0.8631     
Epoch 61/100
7200/7200 [==============================] - 2s - loss: 0.3260 - acc: 0.8644     
Epoch 62/100
7200/7200 [==============================] - 2s - loss: 0.3353 - acc: 0.8639     
Epoch 62/100
7200/7200 [==============================] - 2s - loss: 0.3247 - acc: 0.8665     
Epoch 63/100
7200/7200 [==============================] - 2s - loss: 0.3355 - acc: 0.8658     
Epoch 63/100
7200/7200 [==============================] - 2s - loss: 0.3261 - acc: 0.8658     
Epoch 64/100
7200/7200 [==============================] - 2s - loss: 0.3362 - acc: 0.8651     
Epoch 64/100
7200/7200 [==============================] - 2s - loss: 0.3362 - acc: 0.8637     
Epoch 65/100
7200/7200 [==============================] - 2s - loss: 0.3249 - acc: 0.8668     
Epoch 66/100
7200/7200 [==============================] - 2s - 

In [48]:
accuracies

array([ 0.85624999,  0.8525    ,  0.88249999,  0.84624999,  0.87124999,
        0.8625    ,  0.85499999,  0.84875   ,  0.84499999,  0.87375   ])

In [49]:
variance

0.01208369232724716

In [50]:
mean


0.85937499422579999